In [4]:
import pandas as pd

df = pd.read_csv("../data/processed/tickets_clean.csv")
df.shape

(20000, 17)

In [2]:
#load model hugginFace

from sentence_transformers import SentenceTransformer


model_name="sentence-transformers/all-MiniLM-L6-v2"

model= SentenceTransformer(model_name)


/Users/miriambenali/Desktop/Project-Simplon/NLP-Classification_Support_Tickets--with-MLOps/myvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#embedding
texts= df["clean_text"].tolist()

embeddings= model.encode(
    texts,
    show_progress_bar=True,
    normalize_embeddings=True
)


Batches: 100%|██████████| 625/625 [01:36<00:00,  6.48it/s]


In [9]:
type(embeddings)


numpy.ndarray

In [11]:
import numpy as np

np.isnan(embeddings).sum()

0

In [13]:
import chromadb

client= chromadb.Client()

#create collection chromadb

collection= client.create_collection(
    name = "ticket_embeddings"
)


In [21]:
#index Embedding

import math

batch_size= 5000
num_batches = math.ceil(len(df)/ batch_size)

for i in range(num_batches):
    start = i * batch_size
    end = (i+1) * batch_size

    collection.add(
        documents= df["clean_text"].tolist()[start:end],
        embeddings= embeddings.tolist()[start:end],
        metadatas= metadatas[start:end],
        ids = ids[start:end],
    )



In [22]:
collection.count()

20000

In [24]:
#save Embeddings

import numpy as np

np.save("../data/processed/embeddings.npy", embeddings)

Conclusion:

Nous avons généré des embeddings sémantiques à l’aide d’un modèle Hugging Face et les avons indexés dans ChromaDB afin de permettre une exploitation efficace pour la classification et l’analyse ultérieure.